In [50]:
import numpy as np
from sklearn import datasets
import tensorflow as tf
from sklearn.semi_supervised import LabelPropagation

In [51]:
# load iris data
iris = datasets.load_iris()

In [52]:
labels = np.copy(iris.target)
data = np.copy(iris.data)

In [53]:
len(labels),len(data)

(150, 150)

In [54]:
# get label 0 and 1, and corresponding data
labels = labels[labels < 2]
data = data[np.where(labels < 2)]

In [55]:
# randomly unlabel points
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(labels)) < 0.3
masked_labels = np.copy(labels)
masked_labels[random_unlabeled_points] = 0.5

In [57]:
label_prop_model = LabelPropagation()
lp = label_prop_model.fit(data, masked_labels)

In [58]:
lp.score(data,labels)

1.0

In [63]:
# Parameters
learning_rate = 0.1
num_steps = 100
batch_size = 5
display_step = 100

# Network Parameters
n_hidden_1 = 3 # 1st layer number of neurons
num_input = len(labels) # MNIST data input (img shape: 28*28)
num_classes = 2 # MNIST total classes (0-9 digits)

In [79]:
def neural_net(f0_dict):
    f0 = f0_dict['labels']
    # Hidden fully connected layer with 3 neurons
    layer_1 = tf.contrib.layers.fully_connected(f0, n_hidden_1)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.contrib.layers.fully_connected(layer_1, num_classes)
    return out_layer

In [80]:
def model_fn(features, labels, mode):
    # Build the neural network
    logits = neural_net(features)

    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [81]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'labels': masked_labels}, y=masked_labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


ValueError: Inputs to `Dense` should have rank >= 2.

In [ ]:

# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'labels': masked_labels}, y=masked_labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])